<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Шаг-1.-Изучение-общей-информации" data-toc-modified-id="Шаг-1.-Изучение-общей-информации-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Шаг 1. Изучение общей информации</a></span></li><li><span><a href="#Шаг-2.-Предобработка-данных" data-toc-modified-id="Шаг-2.-Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Шаг 2. Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Шаг-3.-Ответьте-на-вопросы" data-toc-modified-id="Шаг-3.-Ответьте-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Шаг 3. Ответьте на вопросы</a></span></li><li><span><a href="#Шаг-4.-Общий-вывод" data-toc-modified-id="Шаг-4.-Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Шаг 4. Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучение общей информации
Импортируем необходимые библиотеки

In [2]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

Просмотрим и изученим данные

In [3]:
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В данных содержатся следующие типы переменных:
* Категориальные: education, education_id, family, family_status_id, gender, income_type, purpose
* Количественные: children, dats_employed, dob_years, total_income
* Логические: debt

Посмотрим пропущенные значения

In [4]:
df[df['days_employed'].isna()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Посмотрим уникальные значения для каждого столбца

In [4]:
print('Столбец children', df['children'].unique())
print('Столбец days_employed', df['days_employed'].unique())
print('Столбец dob_years', df['dob_years'].unique())
print('Столбец education', df['education'].unique())
print('Столбец education_id', df['education_id'].unique())
print('Столбец family_status', df['family_status'].unique())
print('Столбец family_status_id', df['family_status_id'].unique())
print('Столбец gender', df['gender'].unique())
print('Столбец income_type', df['income_type'].unique())
print('Столбец purpose', df['purpose'].unique())


Столбец children [ 1  0  3  2 -1  4 20  5]
Столбец days_employed [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
Столбец dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
Столбец education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Столбец education_id [0 1 2 3 4]
Столбец family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Столбец family_status_id [0 1 2 3 4]
Столбец gender ['F' 'M' 'XNA']
Столбец income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Столбец purpose ['покупка жилья' 'приобретение автомобиля' 'дополни

In [5]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


**Вывод**

Данные имеют аномалии в виде:

1. 20  и -1 детей. Первое могло быть связанно с печатной  ошибкой или переводом из рукописного ввода в печатный. Второе готоворит об отсутствии детей, т.к самих значений мало.
2. Отрицательное количество дней работы. Это может быть связано с расчетом количества дней работы с момента первой работы по текущий или последний день работы.
3. Положительное количество дней работы. Скорее всего здесь указано время работы в часах, а не днях. 
4. Наличие дубликатов в столбце. Скорее всего отсутствовали инструкции по заполнению по заполнение. Как следствие имеем дубликаты
5. В столбце dob_years встречается цифра 0. Скорее всего клиент не указал дату рождение
6. В столбцах days_employed и total_income имеются пропуски. Клиент не предоставил данные у стаже и доходе.
7. В столбце gender есть значение XNA. Клиент не указал свой пол.

## Шаг 2. Предобработка данных

### Обработка пропусков

Приведем значения стажа клиентов к общему виду:

In [6]:
def converse_days_employed(row):
    if row > 0:
        row = row / 24
        return row
    elif row < 0:
        row = row * -1
        return row
    else:
        return row

df['days_employed'] = df['days_employed'].apply(converse_days_employed)    

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посчитаем средний стаж всех клиентов и заменим пропуски на это значение

In [7]:
avg_days_employed = df['days_employed'].mean()
avg_days_employed


4641.641176180656

In [8]:
df['days_employed'] = df['days_employed'].fillna(avg_days_employed)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Далее посчитаем средний доход по каждому из income_type и заменим этими данными пропуски

In [9]:
retiree = df[df['income_type'] == 'пенсионер']
retire_avg_income = retiree['total_income'].mean()
df[df['income_type'] == 'пенсионер'] = df[df['income_type'] == 'пенсионер'].fillna(retire_avg_income)

employee = df[df['income_type'] == 'сотрудник']
employee_avg_income = employee['total_income'].mean()
df[df['income_type'] == 'сотрудник'] = df[df['income_type'] == 'сотрудник'].fillna(employee_avg_income)

companion = df[df['income_type'] == 'компаньон']
companion_avg_income = companion['total_income'].mean()
df[df['income_type'] == 'компаньон'] = df[df['income_type'] == 'компаньон'].fillna(companion_avg_income)

servant = df[df['income_type'] == 'госслужащий']
servant_avg_income = servant['total_income'].mean()
df[df['income_type'] == 'госслужащий'] = df[df['income_type'] == 'госслужащий'].fillna(servant_avg_income)

unemployed = df[df['income_type'] == 'безработный']
unemployed_avg_income = unemployed['total_income'].mean()
df[df['income_type'] == 'безработный'] = df[df['income_type'] == 'безработный'].fillna(unemployed_avg_income)

businessman = df[df['income_type'] == 'предприниматель']
businessman_avg_income = businessman['total_income'].mean()
df[df['income_type'] == 'предприниматель'] = df[df['income_type'] == 'предприниматель'].fillna(businessman_avg_income)

student = df[df['income_type'] == 'студент']
student_avg_income = student['total_income'].mean()
df[df['income_type'] == 'студент'] = df[df['income_type'] == 'студент'].fillna(student_avg_income)

decree = df[df['income_type'] == 'в декрете']
decree_avg_income = decree['total_income'].mean()
df[df['income_type'] == 'в декрете'] = df[df['income_type'] == 'в декрете'].fillna(decree_avg_income)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Нулевое значение в столбце dob_years**

Заменим нулевое значение на средний возраст всех клиентов в таблице, т.к для проверки гипотез возраст значения не имеет.

In [10]:
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'][df['dob_years'] != 0].mean())
df['dob_years'][df['dob_years'] == 0].count()

0

In [1]:
df.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')

NameError: name 'df' is not defined

**Значение 20 и -1 в столбце children**

20 заменим на 2, вероятно это ошибка перевода. -1 заменим на 1.

In [13]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)
print(df['children'][df['children'] == 20].count())
print(df['children'][df['children'] == -1].count())

0
0


**XNA в столбце gender**

Заменим на М, пол для проверки гипотезы не важен.

In [14]:
df.loc[10701, 'gender'] = 'M'
df.loc[10701, 'gender']

'M'

**Вывод**

Исправили ошибки и аномалии в данных:

1. Убрали отрацательные и не реально большие значения в стаже клиентов.
2. Заполнили пропуски в стаже средними значениями по датасету, т.к стаж для проверки гипотез не важен
3. Заполнили пропуски в доходе клиентов средними значениями по типу дохода.
4. Исправили ошибки в количестве детей.
5. Исправили пропуск в столбце gender.

### Замена типа данных

Заменим дробные числа на целые

In [15]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Данные в столбцах days_employed и total_income теперь целые

### Обработка дубликатов
Еще раз посмотрим на уникальные значения в столбце education

In [16]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Удалим дубликаты с разным регистром

In [17]:
df['education'] = df['education'].str.lower()
print(df['education'].value_counts())
print(df['education_id'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


**Вывод**

Дубликаты исчезли и теперь количество уникальных значений в столбце education соотвествует количеству значений в столбце education_id

### Лемматизация

В столбце purpose много повторяющихся по смыслу значений, но написанны по разному. 
Скорее всего это из-за того, что их писали клиенты в свободной форме.

In [18]:
purpose = " ".join(df['purpose'].unique())
lemmas = m.lemmatize(purpose)
lemmas
print(Counter(lemmas))
#purpose_unique = df['purpose'].unique()
#purpose_set = []

#for i in purpose_unique:
    #print(m.lemmatize(i))


Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


С помощью лемматизации стало отчетливо видно 5 основных целей кредита:

* покупка автомобиля
* покупка жилья
* оплата обучения
* покупка коммерческой недвижимости
* свадьба

In [19]:
def purpose_change(purpose):
    if 'авто' in purpose:
        return 'покупка автомобиля'
    elif 'жиль' in purpose or 'недв' in purpose:
        return 'покупка недвижимости'
    #elif 'недв' in purpose:
        #return 'покупка недвижимости'
    elif 'свадь' in purpose:
        return 'свадьба'
    else:
        return 'оплата образования'
    
df['purpose'] = df['purpose'].apply(purpose_change)
df['purpose'].value_counts()

покупка недвижимости    10840
покупка автомобиля       4315
оплата образования       4022
свадьба                  2348
Name: purpose, dtype: int64

**Вывод**

### Категоризация данных

Упростим проверку гипотез

Создадим категорию семьи

In [20]:
def family_category(row):
    if row == 0:
        return 'нет детей'
    elif row < 3:
        return 'небольшая семья'
    else:
        return 'многодетная семья'
    
df['family_category'] = df['children'].apply(family_category)
df['family_category'].value_counts()

нет детей            14149
небольшая семья       6996
многодетная семья      380
Name: family_category, dtype: int64

Далее категоризируем уровень дохода клиентов:


In [21]:
def income_category(row):
    if row < 100000:
        return 'доход до 100 тысяч'
    if row <= 150000:
        return 'доход от 100 до 150 тысяч'
    if row <= 200000:
        return 'доход от 150 до 200 тысяч'
    else:
        return 'доход свыше 200 тысяч'
    
df['income_category'] = df['total_income'].apply(income_category)    
df['income_category'].value_counts()


доход от 100 до 150 тысяч    6117
доход свыше 200 тысяч        5575
доход от 150 до 200 тысяч    5370
доход до 100 тысяч           4463
Name: income_category, dtype: int64

**Вывод**

Подготовили данные для проверки гипотез, а именно: создали категории по размеру семьи и уровням дохода

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
pivot_count = pd.pivot_table(df, index='family_category', values='debt', aggfunc='count')
pivot_sum = pd.pivot_table(df, index='family_category', values='debt', aggfunc='sum')
debt_by_children = pivot_count
debt_by_children['debt_not_returned'] = pivot_sum['debt']
debt_by_children['returned_debt'] = debt_by_children['debt'] - debt_by_children['debt_not_returned']
debt_by_children['ratio'] = debt_by_children['debt_not_returned'] / debt_by_children['debt']
debt_by_children

,debt,debt_not_returned,returned_debt,ratio
family_category,,,,
многодетная семья,380,31,349,0.081579
небольшая семья,6996,647,6349,0.092481
нет детей,14149,1063,13086,0.075129


**Вывод**
Видим зависимость между наличием детей и возвратом кредита в срок:
* Самый маленький процент невозврата в срок - 7,5% в семьях без детей
* На втором месте - многодетные семьи - е возвращают в срок 8,2%
* Самые ненадежные клиенты, где 1-2 детей, 9,2% не возвращают кредит в срок

In [26]:
pivot_count = pd.pivot_table(df, index='family_status', values='debt', aggfunc='count')
pivot_sum = pd.pivot_table(df, index='family_status', values='debt', aggfunc='sum')
debt_by_family = pivot_count
debt_by_family['debt_not_returned'] = pivot_sum['debt']
debt_by_family['returned_debt'] = debt_by_family['debt'] - debt_by_family['debt_not_returned']
debt_by_family['ratio'] = debt_by_family['debt_not_returned'] / debt_by_family['debt']
debt_by_family

,debt,debt_not_returned,returned_debt,ratio
family_status,,,,
Не женат / не замужем,2813,274,2539,0.097405
в разводе,1195,85,1110,0.071130
вдовец / вдова,960,63,897,0.065625
гражданский брак,4177,388,3789,0.092890
женат / замужем,12380,931,11449,0.075202


**Вывод**
Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Самые надежные - вдовцы и вдовы - 6,6%
* Следом идут люди в разводе - 7,1%
* На третьем месте люди в браке - 7,5%
* На предпоследнем месте гражданский брак - 7,3%
* И самые не надежные клиенты - не женатые/не замужем - 9,7%

In [27]:
pivot_count = pd.pivot_table(df, index='income_category', values='debt', aggfunc='count')
pivot_sum = pd.pivot_table(df, index='income_category', values='debt', aggfunc='sum')
debt_by_income = pivot_count
debt_by_income['debt_not_returned'] = pivot_sum['debt']
debt_by_income['returned_debt'] = debt_by_income['debt'] - debt_by_income['debt_not_returned']
debt_by_income['ratio'] = debt_by_income['debt_not_returned'] / debt_by_income['debt']
debt_by_income

,debt,debt_not_returned,returned_debt,ratio
income_category,,,,
доход до 100 тысяч,4463,354,4109,0.079319
доход от 100 до 150 тысяч,6117,526,5591,0.085990
доход от 150 до 200 тысяч,5370,473,4897,0.088082
доход свыше 200 тысяч,5575,388,5187,0.069596


**Вывод**
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Люди с высоким доходом чаще других возвращают кредиты в срок
* Люди с доходом до 100 тысяч на втором месте
* На третьем месте по надежности люди с доходом от 100 до 150 тысяч в месяц
* Люди с доходом, выше среднего, от 150 до 200 тысяч самые не надежные в плане возврата кредита




In [28]:
pivot_count = pd.pivot_table(df, index='purpose', values='debt', aggfunc='count')
pivot_sum = pd.pivot_table(df, index='purpose', values='debt', aggfunc='sum')
debt_by_purpose = pivot_count
debt_by_purpose['debt_not_returned'] = pivot_sum['debt']
debt_by_purpose['returned_debt'] = debt_by_purpose['debt'] - debt_by_purpose['debt_not_returned']
debt_by_purpose['ratio'] = debt_by_purpose['debt_not_returned'] / debt_by_purpose['debt']
debt_by_purpose

,debt,debt_not_returned,returned_debt,ratio
purpose,,,,
оплата образования,4022,370,3652,0.091994
покупка автомобиля,4315,403,3912,0.093395
покупка недвижимости,10840,782,10058,0.072140
свадьба,2348,186,2162,0.079216


**Вывод**
 - Как разные цели кредита влияют на его возврат в срок?
 * Самый большой процент возврата кредита в срок для цели - покупка недвижимости
 * На втором месте как не странно - кредит на свадьбу
 * На третьем образование
 * И на четвертом месте, самая рисковая цель для банка - покупка автомобиля

## Шаг 4. Общий вывод

Завершили исследование надежности замещиков.

На основе этих данных, мы выяснили:
* Самые надежные клиенты, которые чаще других возвращают кредит в срок - люди, которые берут кредиты для покупки недвижимости(92.8%), самые ненадежные - берут кредит для покупки автомобиля (90,7%)
* Овдовевшие люди надежные клиенты для банка (93.4% таких людей возвращают кредит в срок), а самые не желательные клиенты те, кто никогда не был в браке (90.3%)
* Люди или семьи без детей (9.5%) надежнее тех, семей у которых есть 1 или 2 ребенка (90.75%)
* Чем выше зарплата, тем легче распределить свой бюджет, что влияет на возврат кредита в срок, так люди с зарплатой больше 200 тысяч в месяц чаще других выплачивают кредит вовремя (93%), но клиенты с зарплатой от 150 до 200 тысяч худшие клиенты (91.2%)